In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

import math
import torch
!pip install gpytorch
import gpytorch
from matplotlib import pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.7/156.7 kB 12.9 MB/s eta 0:00:00


In [2]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error

In [3]:
df_clean = pd.read_csv("../input/us-treasury-yield-curve-2010-2018/yield_curve_2010_2018.csv",index_col=0).drop(columns="Date.1")
df_clean.index = pd.DatetimeIndex(df_clean.index)
df_clean = df_clean.sort_index()
print (df_clean.shape)
df_clean.head()

(2002, 11)


,1 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,
2010-01-04,0.05,0.08,0.18,0.45,1.09,1.66,2.65,3.36,3.85,4.60,4.65
2010-01-05,0.03,0.07,0.17,0.41,1.01,1.57,2.56,3.28,3.77,4.54,4.59
2010-01-06,0.03,0.06,0.15,0.40,1.01,1.60,2.60,3.33,3.85,4.63,4.70
2010-01-07,0.02,0.05,0.16,0.40,1.03,1.62,2.62,3.33,3.85,4.62,4.69
2010-01-08,0.02,0.05,0.15,0.37,0.96,1.56,2.57,3.31,3.83,4.61,4.70


# EDA
1. Hurst Component / Autocorelation
2. Persistence Model $\hat{Y}_{t+1} = Y_{t}$

In [4]:
def get_hurst_exponent(time_series, max_lag=20):
    """Returns the Hurst Exponent of the time series"""
    lags = range(2, max_lag)
    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]
    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)
    return reg[0]

In [5]:
pd.DataFrame(df_clean.apply(lambda x: get_hurst_exponent(x.values, 40))).T

,1 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,0.33008,0.460147,0.524739,0.498605,0.443841,0.469158,0.489996,0.490909,0.493751,0.489162,0.470324


### Train/Validation/Test
Train + Validation = 2010 to 2017  
Test = 2018

In [6]:
corr = df_clean.corr()
corr.style.background_gradient()

,1 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
1 Mo,1.000000,0.992347,0.978688,0.955501,0.843656,0.658935,0.337214,0.125799,-0.076583,-0.235160,-0.277069
3 Mo,0.992347,1.000000,0.989907,0.968589,0.857941,0.671222,0.344163,0.129381,-0.077992,-0.239536,-0.281459
6 Mo,0.978688,0.989907,1.000000,0.989115,0.889561,0.700382,0.353527,0.124277,-0.103065,-0.277396,-0.318028
1 Yr,0.955501,0.968589,0.989115,1.000000,0.933835,0.762874,0.418554,0.181262,-0.066014,-0.258921,-0.302840
2 Yr,0.843656,0.857941,0.889561,0.933835,1.000000,0.938132,0.681302,0.456014,0.180907,-0.057811,-0.114843
3 Yr,0.658935,0.671222,0.700382,0.762874,0.938132,1.000000,0.880052,0.702053,0.439560,0.184803,0.119278
5 Yr,0.337214,0.344163,0.353527,0.418554,0.681302,0.880052,1.000000,0.952005,0.794160,0.591062,0.531850
7 Yr,0.125799,0.129381,0.124277,0.181262,0.456014,0.702053,0.952005,1.000000,0.939376,0.803195,0.757528
10 Yr,-0.076583,-0.077992,-0.103065,-0.066014,0.180907,0.439560,0.794160,0.939376,1.000000,0.957449,0.932447
20 Yr,-0.235160,-0.239536,-0.277396,-0.258921,-0.057811,0.184803,0.591062,0.803195,0.957449,1.000000,0.995390


In [7]:
# [["1 Mo","3 Mo","2 Yr", "10 Yr"]]
# [["1 Mo","3 Mo", "6 Mo","1 Yr" , "2 Yr", "2 Yr", "5 Yr" ,"10 Yr", "20 Yr" ,"30 Yr"]]

train_validation = df_clean.loc[:"2016"]
test = df_clean.loc["2017":]

Y_COLS = ["2 Yr", "10 Yr"]

train_validation.shape,test.shape

((1752, 11), (250, 11))

### Persistence Model

In [8]:
result_persist = pd.DataFrame()
tscv = TimeSeriesSplit(n_splits=3,test_size=250)
for i, (train_index, test_index) in enumerate(tscv.split(train_validation)):
    results_i = {}
    print(f"Fold {i}:")
    df_train_i = train_validation.iloc[train_index]
    df_test_i = train_validation.iloc[test_index]
    print(f"Training from {df_train_i.index.min()} to {df_train_i.index.max()}")
    print(f"Testing from {df_test_i.index.min()} to {df_test_i.index.max()}\n")
    #persistence model with rolling one day prediction
    # y_pred_i = pd.DataFrame([df_train_i.iloc[-1].values]*len(df_test_i),index=df_test_i.index,\
    #            columns = df_test_i.columns)
    y_pred_i = df_test_i[Y_COLS].shift(1).copy()
    y_pred_i.iloc[0] = df_train_i[Y_COLS].iloc[-1]
    results_i = {
      "fold":i,
      "test_year":df_test_i.index.year.max()}
    for test_col in Y_COLS:
        results_i[test_col] = {
          "RMSE": mean_squared_error(df_test_i[test_col],y_pred_i[test_col])**0.5,\
          "MAPE": mean_absolute_percentage_error(df_test_i[test_col],y_pred_i[test_col])    
        }
    result_persist = pd.concat([result_persist,pd.DataFrame.from_dict(results_i,orient="columns")])

Fold 0:
Training from 2010-01-04 00:00:00 to 2014-01-02 00:00:00
Testing from 2014-01-03 00:00:00 to 2015-01-02 00:00:00

Fold 1:
Training from 2010-01-04 00:00:00 to 2015-01-02 00:00:00
Testing from 2015-01-05 00:00:00 to 2015-12-31 00:00:00

Fold 2:
Training from 2010-01-04 00:00:00 to 2015-12-31 00:00:00
Testing from 2016-01-04 00:00:00 to 2016-12-30 00:00:00



In [9]:
result_persist.reset_index().drop(columns=["test_year"]).groupby("index").mean()

,fold,2 Yr,10 Yr
index,,,
MAPE,1.0,0.035398,0.017349
RMSE,1.0,0.030475,0.045728


# Use Gaussian Process to model yield curve
## Create lag variables for x

In [10]:
def create_lag_variables(df,y_cols,n_lags=2):
    df1= df.copy()
    col_list = df1.columns
    for lag_i in range(1,n_lags+1):
        df1[[f"{col_i}_lag{lag_i}" for col_i in col_list]] = df1[col_list].shift(lag_i)
    df1 = df1.dropna()
    X = df1.drop(columns=y_cols)
    y = df1[y_cols]
    return X,y

X,y = create_lag_variables(train_validation,Y_COLS,14)
X.shape,y.shape

((1738, 163), (1738, 2))

### Initialise GP model with variable kernel

In [11]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, kernel_func, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=train_y.shape[-1]
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            kernel_func, num_tasks=train_y.shape[-1], rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
def train_gp(train_x, train_y, kernel_func, N_ITER = 50,verbose=True):
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=train_y.shape[-1])
    if type(train_x) != torch.Tensor:
        train_x_tensor = torch.tensor(train_x.values).float()
        train_y_tensor = torch.tensor(train_y.values).float()
    else:
        train_x_tensor = train_x
        train_y_tensor = train_y
    model = MultitaskGPModel(train_x_tensor, train_y_tensor,kernel_func, likelihood)
#     print ("Initialised GP")
    # Find optimal model hyperparameters
    model.train()
    likelihood.train()
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    print ("Training GP")
    for i in range(N_ITER):
        optimizer.zero_grad()
        output = model(train_x_tensor)
        loss = -mll(output, train_y_tensor)
        loss.backward()
        if verbose and ((i%10==0) or (i==N_ITER-1)):
            print('Iter %d/%d - Loss: %.3f' % (i + 1, N_ITER, loss.item()))
        optimizer.step()
    return model,likelihood

def predict_gp(test_x,likelihood,model):
    # Set into eval mode
    model.eval()
    likelihood.eval()
    test_x_tensor = torch.tensor(test_x.values).float()
    # Make predictions
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        predictions = likelihood(model(test_x_tensor))
        mean = predictions.mean
        # lower, upper = predictions.confidence_region()   
    return mean,predictions 

### Train with diff kernels

In [12]:
kernel_repo = {
    #SE
    "kernel1":gpytorch.kernels.RBFKernel(),\
    #SE*Lin
    "kernel2":gpytorch.kernels.RBFKernel()*gpytorch.kernels.LinearKernel(),\
#     #Periodic*Lin
#     "kernel3":gpytorch.kernels.PeriodicKernel()*gpytorch.kernels.LinearKernel(),\
#     #SE*Lin + RQ
#     "kernel4":gpytorch.kernels.RBFKernel()*gpytorch.kernels.LinearKernel() + gpytorch.kernels.RQKernel(),\
#     #SE*Lin + RQ
#     "kernel5":gpytorch.kernels.RBFKernel()*gpytorch.kernels.PeriodicKernel() + gpytorch.kernels.RQKernel()
}

In [13]:
def get_kernel_cv_result_raw(kernel_j, N_TRIALS=10, N_TRAINING_ITER=50, verbose=False):
    result_j = pd.DataFrame()
    model_dict_j = {}
    for trial in range(N_TRIALS):
        for i, (train_index, test_index) in enumerate(tscv.split(X)):
            print(f"Sample {trial}, Fold {i}:")
            train_xi,train_yi = X.iloc[train_index],y.iloc[train_index]
            test_xi,test_yi = X.iloc[test_index],y.iloc[test_index]
#             if verbose:
#                 print(f"Training from {train_xi.index.min()} to {train_xi.index.max()}")
            model_i,lik_i = train_gp(train_xi,train_yi,kernel_j,N_TRAINING_ITER,verbose)
            model_dict_j[trial,i,"model"]=model_i
            if verbose:
                print(f"Testing from {test_xi.index.min()} to {test_xi.index.max()}\n")
            mean_i,pred_i = predict_gp(test_xi,lik_i,model_i)
            model_dict_j[trial,i,"prediction"]=pred_i
            results_i = {
              "trial":trial,
              "fold":i,
              "test_year":test_xi.index.year.max()}
            y_pred_i = pd.DataFrame(mean_i,index=test_yi.index,columns = test_yi.columns)
            for test_col in test_yi.columns:
                results_i[test_col] = {
                  "RMSE": mean_squared_error(test_yi[test_col],y_pred_i[test_col])**0.5,\
                  "MAPE": mean_absolute_percentage_error(test_yi[test_col],y_pred_i[test_col])    
                }
            result_j = pd.concat([result_j,pd.DataFrame.from_dict(results_i,orient="columns")])
    return (result_j,model_dict_j)

In [14]:
result_test,model_test = get_kernel_cv_result_raw(gpytorch.kernels.RBFKernel(),3,10,True)
results_trials = result_test.loc["RMSE"].reset_index().drop(columns=["test_year","fold"]).groupby(["trial","index"]).mean()
results_trials["err_avg"] = results_trials.mean(axis=1)
results_trials["rnk_"] = results_trials["err_avg"].rank()
results_trials.loc[results_trials.rnk_==1]

Sample 0, Fold 0:
Training GP
Iter 1/10 - Loss: 1.396
Iter 10/10 - Loss: 0.872
Testing from 2014-01-03 00:00:00 to 2015-01-02 00:00:00

Sample 0, Fold 1:
Training GP
Iter 1/10 - Loss: 1.184
Iter 10/10 - Loss: 0.783
Testing from 2015-01-05 00:00:00 to 2015-12-31 00:00:00

Sample 0, Fold 2:
Training GP
Iter 1/10 - Loss: 1.159
Iter 10/10 - Loss: 0.781
Testing from 2016-01-04 00:00:00 to 2016-12-30 00:00:00

Sample 1, Fold 0:
Training GP
Iter 1/10 - Loss: 1.134
Iter 10/10 - Loss: 0.763
Testing from 2014-01-03 00:00:00 to 2015-01-02 00:00:00

Sample 1, Fold 1:
Training GP
Iter 1/10 - Loss: 1.123
Iter 10/10 - Loss: 0.752
Testing from 2015-01-05 00:00:00 to 2015-12-31 00:00:00

Sample 1, Fold 2:
Training GP
Iter 1/10 - Loss: 1.116
Iter 10/10 - Loss: 0.750
Testing from 2016-01-04 00:00:00 to 2016-12-30 00:00:00

Sample 2, Fold 0:
Training GP
Iter 1/10 - Loss: 1.115
Iter 10/10 - Loss: 0.754
Testing from 2014-01-03 00:00:00 to 2015-01-02 00:00:00

Sample 2, Fold 1:
Training GP
Iter 1/10 - Loss: 

,,2 Yr,10 Yr,err_avg,rnk_
trial,index,,,,
2,RMSE,0.118492,0.071385,0.094939,1.0


In [15]:
all_results = {}
all_results_df = pd.DataFrame()
for kernel_name,kernel_func_j in kernel_repo.items():
    print (f"Using {kernel_name}")
    kernel_result,kernel_models = get_kernel_cv_result_raw(kernel_func_j,10,200,True)
    kernel_results_trials = kernel_result.loc["RMSE"].reset_index().drop(columns=["fold","test_year"]).groupby(["trial","index"]).mean()
    kernel_results_trials["err_avg"] = kernel_results_trials.mean(axis=1)
    kernel_results_trials["rnk_"] = kernel_results_trials["err_avg"].rank()
    all_results[kernel_name] = (kernel_results_trials,kernel_models)
    result_i = kernel_results_trials.copy()
    result_i = result_i.loc[result_i.rnk_==1]
    result_i["kernel"] = kernel_name
    all_results_df = pd.concat([all_results_df,result_i])

Using kernel1
Sample 0, Fold 0:
Training GP
Iter 1/200 - Loss: 1.342
Iter 11/200 - Loss: 0.805
Iter 21/200 - Loss: 0.295
Iter 31/200 - Loss: -0.227
Iter 41/200 - Loss: -0.741
Iter 51/200 - Loss: -1.229
Iter 61/200 - Loss: -1.669
Iter 71/200 - Loss: -2.032
Iter 81/200 - Loss: -2.297
Iter 91/200 - Loss: -2.471
Iter 101/200 - Loss: -2.573
Iter 111/200 - Loss: -2.635
Iter 121/200 - Loss: -2.674
Iter 131/200 - Loss: -2.700
Iter 141/200 - Loss: -2.718
Iter 151/200 - Loss: -2.732
Iter 161/200 - Loss: -2.744
Iter 171/200 - Loss: -2.753
Iter 181/200 - Loss: -2.760
Iter 191/200 - Loss: -2.767
Iter 200/200 - Loss: -2.772
Testing from 2014-01-03 00:00:00 to 2015-01-02 00:00:00

Sample 0, Fold 1:
Training GP
Iter 1/200 - Loss: 1.102
Iter 11/200 - Loss: 0.700
Iter 21/200 - Loss: 0.240
Iter 31/200 - Loss: -0.253
Iter 41/200 - Loss: -0.755
Iter 51/200 - Loss: -1.245
Iter 61/200 - Loss: -1.705
Iter 71/200 - Loss: -2.106
Iter 81/200 - Loss: -2.415
Iter 91/200 - Loss: -2.615
Iter 101/200 - Loss: -2.727
I

In [16]:
result_i = kernel_results_trials.copy()
result_i = result_i.loc[result_i.rnk_==1.0]
result_i

,,2 Yr,10 Yr,err_avg,rnk_
trial,index,,,,
7,RMSE,0.012791,0.008976,0.010884,1.0


In [17]:
all_results_df

,,2 Yr,10 Yr,err_avg,rnk_,kernel
trial,index,,,,,
4,RMSE,0.016054,0.010183,0.013118,1.0,kernel1
7,RMSE,0.012791,0.008976,0.010884,1.0,kernel2


In [18]:
all_results_df.to_csv("all_results_df.csv")

In [19]:
# import pickle

# with open('all_results.pickle', 'wb') as handle:
#     pickle.dump(all_results, handle)